In [11]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np

In [12]:
def calculate_angle(center_landmark, start_landmark, end_landmark):

    center = np.array(center_landmark)
    start = np.array(start_landmark)
    end = np.array(end_landmark)
    

    vector1 = start - center
    vector2 = end - center
    

    angle_radians = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
    

    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees


def draw_selected_landmarks_on_image(rgb_image, landmarks, selected_landmarks, connections):

    annotated_image = np.copy(rgb_image)


    for index in selected_landmarks:
        landmark = landmarks.landmark[index]
        x, y, z = landmark.x, landmark.y, landmark.z

        x_pixel, y_pixel = int(x * annotated_image.shape[1]), int(y * annotated_image.shape[0])
        cv2.circle(annotated_image, (x_pixel, y_pixel), radius=5, color=(0, 255, 0), thickness=-1)
    for connection in connections:
        start_index, end_index = connection
        start_landmark = landmarks.landmark[start_index]
        end_landmark = landmarks.landmark[end_index]
        start_x, start_y = int(start_landmark.x * annotated_image.shape[1]), int(start_landmark.y * annotated_image.shape[0])
        end_x, end_y = int(end_landmark.x * annotated_image.shape[1]), int(end_landmark.y * annotated_image.shape[0])
        cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
            

    return annotated_image

In [13]:
angles_list=[]

In [14]:
mp_pose = mp.solutions.pose
selected_landmarks_indices = [16,14,12,24,26,28,32]
selected_landmarks_connections = [(16,14),(14,12),(24,26),(26,28),(12,24),(24,28),(28,32)]

In [6]:
with mp_pose.Pose(
    static_image_mode=False, min_detection_confidence=0.5, model_complexity=1
) as pose:
    video_path = r"C:\Users\Wajdan\Desktop\incorrect hip.mp4"
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(frame_rgb)

        landmark_12 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[2]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[2]].y,
        )

        landmark_24 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[3]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[3]].y,
        )

        landmark_26 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[4]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[4]].y,
        )

        angle = calculate_angle(landmark_24, landmark_12, landmark_26)

        angles_list.append((angle, 0))

    cap.release()

    cv2.destroyAllWindows()

In [7]:
df = pd.DataFrame(angles_list, columns=["angle", "label"])

In [8]:
df[df['label']==0].mean()

angle    122.914607
label      0.000000
dtype: float64

In [9]:
df[df['label']==1].mean()
# df = df.drop(df[(df['label'] == 1) & (df['angle'] < 170)].index)

angle    175.326053
label      1.000000
dtype: float64

In [10]:
df.to_csv(
    r"C:\Users\Wajdan\Desktop\Islah.AI-FinalYearProject-ComputerVision\qiyam\hip-train.csv",
    mode="a",
    # header=False,
    index=False,
)

In [18]:
with mp_pose.Pose(
    static_image_mode=False, min_detection_confidence=0.5, model_complexity=1
) as pose:
    video_path = r"C:\Users\Wajdan\Desktop\correct qiyam.mp4"
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(frame_rgb)

        landmark_16 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[0]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[0]].y,
        )

        landmark_14 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[1]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[1]].y,
        )

        landmark_12 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[2]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[2]].y,
        )

        landmark_24 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[3]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[3]].y,
        )

        landmark_26 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[4]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[4]].y,
        )
        landmark_28 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[5]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[5]].y,
        )
        landmark_32 = (
            results.pose_landmarks.landmark[selected_landmarks_indices[6]].x,
            results.pose_landmarks.landmark[selected_landmarks_indices[6]].y,
        )
        angle = calculate_angle(landmark_24, landmark_12, landmark_28)

        frame = draw_selected_landmarks_on_image(
            frame,
            results.pose_landmarks,
            selected_landmarks_indices,
            selected_landmarks_connections,
        )

        angle_text = f"{angle:.2f}"

        font = cv2.FONT_HERSHEY_SIMPLEX

        font_scale = 1

        font_thickness = 2

        line_spacing = 10

        text_size = cv2.getTextSize(angle_text, font, font_scale, font_thickness)[0]

        text_width = text_size[0]

        text_height = 10

        text_x = 10

        text_y = text_height + 10

        cv2.rectangle(
            frame,
            (text_x, text_y - text_height),
            (text_x + text_width, text_y),
            (0, 0, 0),
            -1,
        )

        cv2.putText(
            frame,
            angle_text,
            (text_x, text_y),
            font,
            font_scale,
            (0, 0, 255),
            font_thickness,
            cv2.LINE_AA,
        )

        cv2.imshow("Annotated Frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()

    cv2.destroyAllWindows()